In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

##### Naive RAG. Approach 1

In [6]:
# pip install pypdf faiss-cpu
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS

In [7]:
file_paths = os.listdir("files")
file_paths

['NVIDIA-2022-Annual-Report.pdf',
 'NVIDIA-2023-Annual-Report.pdf',
 'NVIDIA-2024-Annual-Report.pdf']

In [8]:
all_pages = []
for file_path in file_paths:
    loader = PyPDFLoader("files/" + file_path)
    pages = loader.load_and_split()
    all_pages += pages

In [9]:
vectorstore = FAISS.from_documents(all_pages, OpenAIEmbeddings())

In [10]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [11]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. "
            "If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise."
        ),
        HumanMessagePromptTemplate.from_template(
            "Question: {question} \n\nContext: {context}\n\n Answer:"
        ),
    ]
)

output_parser = StrOutputParser()

In [13]:
retriever = vectorstore.as_retriever()

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | chat_prompt
    | llm
    | output_parser
)

In [23]:
rag_chain.invoke("What are the Nvidia achievements in 2024?")

'In 2024, Nvidia achieved significant milestones such as tripling Data Center revenue, launching the Arm-based data center CPU Grace, introducing the Blackwell GPU architecture for AI applications, and surpassing $1 billion in Automotive revenue with the NVIDIA DRIVE Orin AI car computer. Nvidia also announced the GeForce RTX 40 SUPER Series family of gaming GPUs and advancements in cloud-native Inference Microservices for AI applications. Additionally, they continued to advance adoption of Omniverse across various industries.'

In [24]:
result = retriever.invoke("What are the Nvidia achievements in 2022?")

In [25]:
result

[Document(id='b936df86-98d6-4c84-a260-41818c18e7b5', metadata={'source': 'files/NVIDIA-2024-Annual-Report.pdf', 'page': 33, 'page_label': '34'}, page_content='Business Highlights\n• NVIDIA accelerated computing reached the tipping point, with Data Center revenue more than tripling in Fiscal \n2024 on higher shipments of the NVIDIA Hopper GPU computing platform and InfiniBand \n• Started shipping our first Arm-based data center CPU – Grace – as part of the GH200 Grace Hopper Superchip, \nand ramping Grace-based products into a new multi-billion dollar product line\n• Launched the Blackwell GPU architecture, powering a wave of upcoming data center-scale platforms for the \ntrillion-parameter-scale AI\n• Introduced NVIDIA Spectrum-X, an accelerated networking platform for building multi-tenant, hyperscale AI \nclouds with Ethernet, and announced the new Quantum-X800 InfiniBand and Spectrum-X800 Ethernet switches\n• Introduced cloud-native NVIDIA Inference Microservices to help developers 

##### Agent RAG With Tools. Approach 2

In [26]:
# pip install pypdf faiss-cpu
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool
from langgraph.prebuilt import create_react_agent

In [27]:
folder_path = "files"
file_paths = os.listdir(folder_path)
file_paths

['NVIDIA-2022-Annual-Report.pdf',
 'NVIDIA-2023-Annual-Report.pdf',
 'NVIDIA-2024-Annual-Report.pdf']

In [28]:
# Initialize a dictionary to store retriever tools
retriever_tools = {}

In [29]:
# Loop through each file and create a separate retriever tool
for file_name in file_paths:

    loader = PyPDFLoader("files/" + file_path)
    pages = loader.load_and_split()

    # Create a vector store for each document
    vector_store = FAISS.from_documents(pages, OpenAIEmbeddings())
    retriever = vector_store.as_retriever()

    # Create a retriever tool with a unique description
    tool_name = f"report_search_{file_name.replace('.pdf', '')}"
    description = f"Search within the document {file_name} for relevant information."

    retriever_tools[file_name] = create_retriever_tool(
        retriever, tool_name, description
    )

In [30]:
retriever_tools

{'NVIDIA-2022-Annual-Report.pdf': Tool(name='report_search_NVIDIA-2022-Annual-Report', description='Search within the document NVIDIA-2022-Annual-Report.pdf for relevant information.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000262F86FACA0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000262FF371790>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x00000262F8863420>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000262FF371790>, search_kwargs={}), document_prompt=PromptTemplate(input_varia

In [31]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [32]:
tools = [retriever_tools[file_name] for file_name in file_paths]

In [33]:
agent_executor = create_react_agent(model, tools)

In [36]:
from langchain_core.messages import HumanMessage

result = agent_executor.invoke(
    {"messages": [HumanMessage(content="What are the achievements of NVIDIA in 2024?")]}
)
result

{'messages': [HumanMessage(content='What are the achievements of NVIDIA in 2024?', additional_kwargs={}, response_metadata={}, id='b129891e-7c09-405f-829b-97dc8cd79151'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SwByJRRXmBZsjxgBXpXto5ot', 'function': {'arguments': '{"query":"achievements"}', 'name': 'report_search_NVIDIA-2024-Annual-Report'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 187, 'total_tokens': 212, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3c29c3f5-964c-41be-a393-f572cc54f04c-0', tool_calls=[{'name': 'report_search_NVIDIA-2024-Annual-Report', 'args': {'query': 'achievements'}, 'id': 'call_Sw

In [37]:
result = agent_executor.invoke(
    {"messages": [HumanMessage(content="What are the achievements of NVIDIA in 2022?")]}
)
result

{'messages': [HumanMessage(content='What are the achievements of NVIDIA in 2022?', additional_kwargs={}, response_metadata={}, id='9a61847f-160b-45fd-afb6-942c4c99bc23'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wnYXzoT2zcpTvcvuSaDw13d3', 'function': {'arguments': '{"query":"achievements"}', 'name': 'report_search_NVIDIA-2022-Annual-Report'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 187, 'total_tokens': 212, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-605167a9-d0c4-40d8-a2bd-b2fcf5d9d017-0', tool_calls=[{'name': 'report_search_NVIDIA-2022-Annual-Report', 'args': {'query': 'achievements'}, 'id': 'call_wn